# <font color='black'>НИС: регрессионный анализ, 2025 </font>
## <font color='black'> Практическое занятие: Гребневая регрессия. Меры качества регрессионной модели. Сравнение альтернативных спецификаций моделей </font>

В рамках данного практического занятия мы рассмотрим реализацию метода гребневой регрессии. Исходные данные:

* lngdp2 - логарифм ВВП на душу населения - зависимая переменная

В качестве предикторов выступают 6 индексов качества гос. управления WGI (Worldwide Governance Indicators)
* va - voice and accountability
* rl - rule and law
* rq - regulatory quality
* gove - government effectiveness
* ps - political stability
* cc - control of corruption

Распределения этих индексов приведено в рамках проекта к стандартному нормальному, поэтому мы не используем дополнительно стандартизацию

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from statsmodels.stats.anova import anova_lm

In [ ]:
dta = pd.read_stata('data_ridge.dta')
dta = dta.dropna()

Оценим модель m1 на исходных данных. Можем ли мы доверять полученным результатам?

In [ ]:
m1 = smf.ols(formula = "lngdp2 ~ va + rl + rq + gove + ps + cc", data = dta).fit()
print(m1.summary())

Выведем корреляционную матрицу для наших предикторов:

In [ ]:
dta[["va", "rl", "rq", "gove", "ps", "cc"]].corr().round(3)

Кроме этого, понять, есть ли у нас свидетельства в пользу сильной мультиколлинеарности, нам помогут коэффициенты VIF (коэффициенты "вздутия" дисперсии). Проинтерпретируйте полученные результаты

In [ ]:
X = dta[["va", "rl", "rq", "gove", "ps", "cc"]]
X = add_constant(X)

vif_data = pd.DataFrame({'variables':X.columns[1:], 'VIF':[variance_inflation_factor(X.values, i+1) for i in range(len(X.columns[1:]))]})
print(vif_data)

 Применим гребневую регрессию для получения более устойчивых результатов. Воспользуемся процедурой кросс-валидации для подбора оптимального параметра регуляризации. Разделим наш массив на 2 подвыборки: тестовую (20% данных) и обучающую (соответственно, 80% данных)

In [ ]:
train, test = train_test_split(dta, test_size = 0.2, random_state = 1)

Для начала используем 1 в качестве параметра регуляризации и выведем оценки коэффициентов в модели гребневой регрессии:

In [ ]:
ridge1 = Ridge(alpha = 1)
ridge1.fit(train[["va", "rl", "rq", "gove", "ps", "cc"]], train["lngdp2"])
ridge1.coef_

Напишем функцию для расчета стандартных ошибок:

In [ ]:
def ridge_se(model, X, y):
    n, p = X.shape
    X_matrix = X.values if hasattr(X, 'values') else X

    y_pred = model.predict(X)
    residuals = y - y_pred
    sigma_sq = np.sum(residuals**2) / (n - p - 1)

    XTX = X_matrix.T @ X_matrix
    lambda_I = model.alpha * np.eye(p)
    inv_matrix = np.linalg.inv(XTX + lambda_I)

    cov_matrix = sigma_sq * inv_matrix @ XTX @ inv_matrix
    se = np.sqrt(np.diag(cov_matrix))

    return se

Рассчитаем стандартные ошибки для оценок коэффициентов нашей первой модели гребневой регрессии:

In [ ]:
X = train[["va", "rl", "rq", "gove", "ps", "cc"]]
y = train['lngdp2']
se1 = ridge_se(ridge1, X, y)

se1

Для удобства представим ниже таблицу, в которой сравним оценки коэффициентов и их значимость в исходной модели и модели гребневой регрессии (с параметром $\alpha$ = 1):

In [ ]:
ridge1_data = pd.DataFrame({'variables':X.columns, 'coef': m1.params[1:], 'se': m1.bse[1:], 't': m1.params[1:]/m1.bse[1:],
                            'coef_ridge':ridge1.coef_, 'se_ridge': se1, 't_ridge': ridge1.coef_/se1})
print(ridge1_data)

Подберем оптимальный параметр регуляризации. Для этого используем k-блочную кросс-валидацию: то есть, массив разбивается на k равных подвыборок, далее проводим для каждого заданного параметра $\alpha$ k итераций: j-ая подвыборка выступает тестовой, остальные подвыборки составляют обучающую. Считаем среднее MSE по k итерациям для каждого значения $\alpha$ и далее останавливаемся на том значении $\alpha$, при котором усредненное MSE принимает минимальное значение

In [ ]:
alphas = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

grid_search = GridSearchCV(ridge1, {'alpha': alphas}, cv = 5)
grid_search.fit(train[["va", "rl", "rq", "gove", "ps", "cc"]], train["lngdp2"])

print("Best Regularization Parameter:", grid_search.best_params_)

In [ ]:
ridge2 = Ridge(alpha = 10)
ridge2.fit(train[["va", "rl", "rq", "gove", "ps", "cc"]], train["lngdp2"])
ridge2.coef_

In [ ]:
se2 = ridge_se(ridge2, X, y)

se2

In [ ]:
ridge2_data = pd.DataFrame({'variables':X.columns, 'coef': m1.params[1:], 'se': m1.bse[1:], 't': m1.params[1:]/m1.bse[1:],
                            'coef_ridge':ridge2.coef_, 'se_ridge': se2, 't_ridge': ridge2.coef_/se2})
print(ridge2_data)

Далее для понимания, можем ли мы обобщать результаты на более широкую выборку, сравним $R^2$ и $MSE$ на тестовой и обучающей подвыборках:

In [ ]:
y_pred_test = ridge2.predict(test[["va", "rl", "rq", "gove", "ps", "cc"]])
mse_test = mean_squared_error(test["lngdp2"], y_pred_test)

In [ ]:
r2_test = r2_score(test["lngdp2"], y_pred_test)

In [ ]:
y_pred_train = ridge2.predict(train[["va", "rl", "rq", "gove", "ps", "cc"]])
mse_train = mean_squared_error(train["lngdp2"], y_pred_train)

In [ ]:
r2_train = r2_score(train["lngdp2"], y_pred_train)

In [ ]:
print(f'R2 обучающая выборка: {r2_train:.3f}\nR2 тестовая выборка: {r2_test:.3f}')
print(f'MSE обучающая выборка: {mse_train:.3f}\nMSE тестовая выборка: {mse_test:.3f}')

Оценим две спецификации модели, вложенные друг в друга - они будут различаться на один параметр. Проследим, как изменяются меры коэффициента детерминации и скорректированного коэффициента детерминации с добавлением предиктора.

$R^2$ скорректированный ($R^2_{adj}$) штрафует модель за "нагруженность", то есть, за дополнительные предикторы. $R^2_{adj}$ рассчитывается по следующей формуле:

$R^2_{adj} = 1 - \frac{RSS \times (N-1)}{TSS \times (N-k-1)}$, где k - это количество предикторов в модели

In [ ]:
m2 = smf.ols(formula = "lngdp2 ~ rl", data = dta).fit(cov_type = "HC3")
print(m2.summary())

In [ ]:
m3 = smf.ols(formula = "lngdp2 ~ rl + va", data = dta).fit(cov_type = "HC3")
print(m3.summary())

In [ ]:
print("Model2 R-squared:", m2.rsquared.round(3), "Model2 R-squared adjusted:", m2.rsquared_adj.round(3))
print("Model3 R-squared:", m3.rsquared.round(3), "Model3 R-squared adjusted:", m3.rsquared_adj.round(3))

В то время как $R^2$ увеличивается с добавлением новых предикторов, $R^2_{adj}$ может и уменьшиться (в случае добавленных незначимых предикторов). Для лучшего понимания выведем отдельно таблицу разложения вариации для m3.

In [ ]:
print(anova_lm(m3))

Важно понять, устойчивы ли показатели качества регрессионной модели. Для того, чтобы результаты были более обоснованы и не опирались лишь на одно разбиение массива на тестовую и обучающую выборки, используем k-блочную кросс-валидацию (поделим данные на 5 фолдов, проведем 5 итераций и усредним полученные результаты: на каждой итерации алгоритма модель обучается на 4 фолдах, тестируется на оставшейся 5-ой части выборки).

Как мы видим, результаты неустойчивы, что вполне объяснимо с учетом наших данных и спецификации модели

In [ ]:
X = dta[["rl", "va"]]
y = dta["lngdp2"]

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=1)
R2 = []

In [ ]:
for fold, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    m1.cv = LinearRegression()
    m1.cv.fit(X_train, y_train)

    y_pred = m1.cv.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    R2.append(r2)

    print("Fold", fold+1, "R2:", r2)

average_R2 = sum(R2) / len(R2)
print("Average R2:", average_R2)

Сравним альтернативные спецификации моделей m3 и m4 при помощи информационных критериев AIC и BIC.

In [ ]:
m4 = smf.ols(formula = "lngdp2 ~ va + ps", data = dta).fit(cov_type = "HC3")
print(m4.summary())

In [ ]:
aic_m3 = m3.aic
aic_m4 = m4.aic

bic_m3 = m3.bic
bic_m4 = m4.bic

print("Model3 AIC:", aic_m3.round(3), "Model4 AIC:", aic_m4.round(3))
print("Model3 BIC:", bic_m3.round(3), "Model4 BIC:", bic_m4.round(3))

In [ ]:
p = len(m3.params)
LL = m3.llf
aicm3 = 2*p - 2*LL
aicm3

In [ ]:
bicm3 = np.log(len(dta))*p - 2*LL
bicm3

Для вложенных моделей мы можем использовать F-test. Статистика для этого теста рассчитывается следующим образом:

$F = \frac{(RSS_{short} - RSS_{long})/Δ df}{RSS_{long}/df_{long}}$

При верной нулевой гипотезе такая статистика имеет распределение Фишера с количеством степеней свободы: $df_{1} = \Delta df$, $df_{2} = df_{long}$

In [ ]:
anovaResults = anova_lm(m2, m3)
print(anovaResults)